In [1]:
import pandas as pd
import gdown

# Load the dataset
output_file = 'mixalldata_clean.csv'
df = pd.read_csv(output_file)

# Display the DataFrame
print(df.head())

   type      sendTime  sender  senderPseudo  messageID  class        posx  \
0     4  72002.302942  130137     101301377  422013806      0  266.982401   
1     4  72003.302942  130137     101301377  422023410      0  266.827208   
2     4  72004.302942  130137     101301377  422032081      0  266.420297   
3     4  72005.302942  130137     101301377  422040712      0  268.912026   
4     4  72006.302942  130137     101301377  422052949      0  268.242276   

        posy  posz    posx_n  ...  aclz    aclx_n    acly_n  aclz_n      hedx  \
0  32.336955   0.0  3.480882  ...   0.0  0.000862  0.000862     0.0 -0.102790   
1  34.624145   0.0  3.546261  ...   0.0  0.000107  0.001040     0.0 -0.099856   
2  38.836461   0.0  3.544045  ...   0.0  0.000172  0.001661     0.0 -0.099856   
3  45.414229   0.0  3.340080  ...   0.0  0.000171  0.001654     0.0 -0.100172   
4  53.729986   0.0  3.328872  ...   0.0  0.000193  0.001852     0.0 -0.097105   

       hedy  hedz     hedx_n     hedy_n  hedz_n  


In [2]:
import pandas as pd

# Load the dataset
file_path = 'mixalldata_clean.csv'
df = pd.read_csv(file_path)

# Specify numeric columns
numeric_columns = [
    'sendTime', 'sender', 'senderPseudo', 'posx', 'posy', 'posz', 'posx_n', 'posy_n', 'posz_n',
    'spdx', 'spdy', 'spdz', 'spdx_n', 'spdy_n', 'spdz_n',
    'aclx', 'acly', 'aclz', 'aclx_n', 'acly_n', 'aclz_n',
    'hedx', 'hedy', 'hedz', 'hedx_n', 'hedy_n', 'hedz_n', 'class'
]
df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Sort by 'sender' and 'sendTime' for time-sequential ordering within each sender
df = df.sort_values(by=['senderPseudo', 'sendTime']).reset_index(drop=True)

# Separate filtered records into misbehavior and normal classes
misbehavior_records = df[df['class'] != 0]
normal_records = df[df['class'] == 0]

# Sample 2000 records per misbehavior class, ensuring each sender has at least 5 occurrences
misbehavior_sample = (
    misbehavior_records.groupby('class')
    .apply(lambda x: x.head(2500))
    .reset_index(drop=True)
)

# Calculate the remaining sample size needed from normal records to reach 100,000 total
normal_sample_size = 125000 - len(misbehavior_sample)

# Sample normal records while ensuring each sender occurs at least 5 times
normal_sample = (
    normal_records.groupby('sender')
    .apply(lambda x: x.head(5))  # Take at least 5 records per sender
    .reset_index(drop=True)
)

# If additional normal records are needed, sample from remaining normal records
additional_normal_records_needed = normal_sample_size - len(normal_sample)
if additional_normal_records_needed > 0:
    remaining_normal_sample = (
        normal_records[~normal_records.index.isin(normal_sample.index)]
        .sample(n=additional_normal_records_needed, random_state=42)
    )
    normal_sample = pd.concat([normal_sample, remaining_normal_sample])

# Combine misbehavior and normal samples, and sort by 'sendTime' for a sequential dataset
final_df = pd.concat([normal_sample, misbehavior_sample]).sort_values(by='sendTime').reset_index(drop=True)

# Display final class distribution to verify balance and sender occurrence condition
print("Class distribution in final dataset:")
print(final_df['class'].value_counts())

# Verify each sender appears at least 5 times in the final dataset
sender_occurrences = final_df['sender'].value_counts()
print("All senders have at least 5 occurrences:", (sender_occurrences >= 5).all())

Class distribution in final dataset:
0     85989
18     2500
1      2500
2      2500
3      2500
4      2500
5      2500
6      2500
7      2500
8      2500
19     2500
10     2500
11     2500
12     2500
13     2500
14     2500
15     2500
16     2500
17     2500
9      2500
Name: class, dtype: int64
All senders have at least 5 occurrences: False


In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

lr = LogisticRegression(random_state=42)

lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("Logistic Regression Accuracy:", accuracy)
print("Logistic Regression Precision:", precision)
print("Logistic Regression Recall:", recall)
print("Logistic Regression F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Logistic Regression Accuracy: 0.7744774889504832
Logistic Regression Precision: 0.9309674282936315
Logistic Regression Recall: 0.40033448311905506
Logistic Regression F1-score: 0.5599005920619838

Training time: 0.271917 seconds


In [4]:
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = lr.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Logistic Regression Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
Logistic Regression Predicted label (class): 0

Prediction time: 0.002001 seconds


In [5]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1)
    true_label = Y_test.iloc[random_index]

    predicted_label = lr.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Logistic Regression")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Logistic Regression
Classification Accuracy: 0.73
Classification Precision (avg): 0.80
Classification Recall (avg): 0.66
Classification F1-Score (avg): 0.65
Average Prediction Time: 0.000072 seconds
Predictions with time >= 0.1s: 0 out of 500


In [6]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
import time

start_time = time.time()
warnings.filterwarnings("ignore", category=FutureWarning)

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

n_neighbor_amount = 10

knn = KNeighborsClassifier(n_neighbors=n_neighbor_amount)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("KNN Accuracy when n_neighbors =", n_neighbor_amount, ":", accuracy)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("KNN Precision:", precision)
print("KNN Recall:", recall)
print("KNN F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

KNN Accuracy when n_neighbors = 10 : 0.9244512697580343
KNN Precision: 0.9548192771084337
KNN Recall: 0.8283683495348594
KNN F1-score: 0.8871103151060614

Training time: 1.988935 seconds


In [7]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = knn.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"KNN Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
KNN Predicted label (class): 0

Prediction time: 0.004596 seconds


In [8]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = knn.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("KNN")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

KNN
Classification Accuracy: 0.93
Classification Precision (avg): 0.94
Classification Recall (avg): 0.91
Classification F1-Score (avg): 0.92
Average Prediction Time: 0.002569 seconds
Predictions with time >= 0.1s: 0 out of 500


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=20, random_state=42)

rf.fit(X_train, Y_train)

Y_pred = rf.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("Random Forest Accuracy:", accuracy)
print("Random Forest Precision:", precision)
print("Random Forest Recall:", recall)
print("Random Forest F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Random Forest Accuracy: 0.9758034309686119
Random Forest Precision: 0.9786457774439318
Random Forest Recall: 0.9532768893069928
Random Forest F1-score: 0.9657947686116701

Training time: 8.627754 seconds


In [11]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = rf.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Random Forest Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
Random Forest Predicted label (class): 0

Prediction time: 0.009824 seconds


In [12]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = rf.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Random Forest")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Random Forest
Classification Accuracy: 0.99
Classification Precision (avg): 0.99
Classification Recall (avg): 0.98
Classification F1-Score (avg): 0.98
Average Prediction Time: 0.003156 seconds
Predictions with time >= 0.1s: 0 out of 500


In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train, Y_train)

Y_pred = dt.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("Decision Tree Accuracy:", accuracy)
print("Decision Tree Precision:", precision)
print("Decision Tree Recall:", recall)
print("Decision Tree F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Decision Tree Accuracy: 0.9634429545284291
Decision Tree Precision: 0.949075036374974
Decision Tree Recall: 0.9494697442295695
Decision Tree F1-score: 0.9492723492723494

Training time: 3.601831 seconds


In [40]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = dt.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Decision Tree Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 1
Predicted label (class): 1

Prediction time: 0.006588 seconds


In [14]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = dt.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Decision Tree")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Decision Tree
Classification Accuracy: 0.97
Classification Precision (avg): 0.97
Classification Recall (avg): 0.96
Classification F1-Score (avg): 0.97
Average Prediction Time: 0.001464 seconds
Predictions with time >= 0.1s: 0 out of 500


In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.31, random_state=42)

nb = GaussianNB()

nb.fit(X_train, Y_train)

Y_pred = nb.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("Naive Bayes Accuracy:", accuracy)
print("Naive Bayes Precision:", precision)
print("Naive Bayes Recall:", recall)
print("Naive Bayes F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Naive Bayes Accuracy: 0.8228215166014209
Naive Bayes Precision: 0.8524729156853509
Naive Bayes Recall: 0.6107998650016875
Naive Bayes F1-score: 0.7116791191506096

Training time: 0.097292 seconds


In [24]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = nb.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Naive Bayes Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 1
Naive Bayes Predicted label (class): 1

Prediction time: 0.008000 seconds


In [25]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = nb.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Naive Bayes")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Naive Bayes
Classification Accuracy: 0.82
Classification Precision (avg): 0.84
Classification Recall (avg): 0.78
Classification F1-Score (avg): 0.79
Average Prediction Time: 0.001722 seconds
Predictions with time >= 0.1s: 0 out of 500


In [26]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42,
                    max_iter=100, tol=0.001)

mlp.fit(X_train, Y_train)

Y_pred = mlp.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("MLP Accuracy:", accuracy)
print("MLP Precision:", precision)
print("MLP Recall:", recall)
print("MLP F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

MLP Accuracy: 0.9679751292231628
MLP Precision: 0.9814323607427056
MLP Recall: 0.9281906553778614
MLP F1-score: 0.9540692989524576

Training time: 96.866027 seconds


In [27]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = X_test[random_index].reshape(1, -1)
true_label = Y_test.iloc[random_index]

predicted_label = mlp.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"MLP Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
MLP Predicted label (class): 0

Prediction time: 0.001216 seconds


In [29]:
import numpy as np
import time
import pandas as pd
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if isinstance(Y_test, np.ndarray) and Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1)
    true_label = Y_test.iloc[random_index] if isinstance(Y_test, pd.Series) else Y_test[random_index]

    predicted_label = mlp.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("MLP Classification Results")
print(f"Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Precision (avg): {avg_precision:.2f}")
print(f"Recall (avg): {avg_recall:.2f}")
print(f"F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

MLP Classification Results
Accuracy: 0.97
Precision (avg): 0.98
Recall (avg): 0.97
F1-Score (avg): 0.97
Average Prediction Time: 0.000140 seconds
Predictions with time >= 0.1s: 0 out of 500


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import time

start_time = time.time()
df = pd.read_csv("mixalldata_clean.csv")

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class'] != 0).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(len(features),)),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

loss, accuracy = model.evaluate(X_test, Y_test)

Y_test = np.squeeze(Y_test)
Y_pred = np.squeeze(Y_pred)

precision = precision_score(Y_test, Y_pred)
recall = recall_score(Y_test, Y_pred)
f1 = f1_score(Y_test, Y_pred)

end_time = time.time()
elapsed_time = end_time - start_time

print("ANN Accuracy:", accuracy)
print("ANN Precision:", precision)
print("ANN Recall:", recall)
print("ANN F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.7384 - loss: 0.8244 - val_accuracy: 0.8233 - val_loss: 0.3736
Epoch 2/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8371 - loss: 0.3623 - val_accuracy: 0.8615 - val_loss: 0.3214
Epoch 3/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8626 - loss: 0.3130 - val_accuracy: 0.8671 - val_loss: 0.3288
Epoch 4/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.8828 - loss: 0.2746 - val_accuracy: 0.8919 - val_loss: 0.2606
Epoch 5/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9018 - loss: 0.2458 - val_accuracy: 0.9082 - val_loss: 0.2255
Epoch 6/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9136 - loss: 0.2237 - val_accuracy: 0.9169 - val_loss: 0.2103
Epoch 7/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9201 - loss: 0.2105 - val_accuracy: 0.9297 - val_loss: 0.1936
Epoch 8/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 5s 2ms/step - accuracy: 0.9259 - loss: 0.2042 - 

In [20]:
import pandas as pd
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))
sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_prob = model.predict(sample)[0, 0]
predicted_label = int(predicted_prob > 0.5)

print("\nRandomly selected sample:")
print(f"True label: {true_label}")
print(f"ANN Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

Randomly selected sample:
True label: 0
ANN Predicted label (class): 0

Prediction time: 0.073082 seconds


In [22]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

for i in range(total_samples_processed):
    start_time = time.time()

    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_prob = model.predict(sample)[0, 0]
    predicted_label = int(predicted_prob > 0.5)

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("\nANN Model")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━

In [35]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input

start_time = time.time()

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features].values
Y = (final_df['class'] != 0).astype(int)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=15, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("DNN Accuracy:", accuracy)
print("DNN Precision:", precision)
print("DNN Recall:", recall)
print("DNN F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8834 - loss: 0.2984 - val_accuracy: 0.9323 - val_loss: 0.1873
Epoch 2/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9261 - loss: 0.2079 - val_accuracy: 0.9388 - val_loss: 0.1776
Epoch 3/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9327 - loss: 0.1920 - val_accuracy: 0.9422 - val_loss: 0.1684
Epoch 4/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9358 - loss: 0.1855 - val_accuracy: 0.9455 - val_loss: 0.1613
Epoch 5/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9403 - loss: 0.1760 - val_accuracy: 0.9479 - val_loss: 0.1559
Epoch 6/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - accuracy: 0.9415 - loss: 0.1702 - val_accuracy: 0.9475 - val_loss: 0.1547
Epoch 7/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.9435 - loss: 0.1674 - val_accuracy: 0.9514 - val_loss: 0.1469
Epoch 8/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.9454 - loss: 0.1643 - 

In [41]:
import pandas as pd
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))
sample = X_test[random_index].reshape(1, -1)
true_label = np.array(Y_test)[random_index]

predicted_prob = model.predict(sample)[0, 0]
predicted_label = int(predicted_prob > 0.5)

print("\nRandomly selected sample:")
print(f"True label: {true_label}")
print(f"DNN Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step

Randomly selected sample:
True label: 0
DNN Predicted label (class): 0

Prediction time: 0.085998 seconds


In [43]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

for i in range(total_samples_processed):
    start_time = time.time()

    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1)
    true_label = np.array(Y_test)[random_index]

    predicted_prob = model.predict(sample)[0, 0]
    predicted_label = int(predicted_prob > 0.5)

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("\nDNN Model")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

In [46]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, Input

start_time = time.time()

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features].values
Y = (final_df['class'] != 0).astype(int)  # Ensure binary labels (0 or 1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = X.reshape(X.shape[0], 1, X.shape[1])  # Reshape for LSTM (samples, timesteps=1, features)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(1, X_train.shape[2])),
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(32, dropout=0.2, recurrent_dropout=0.1),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=15, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted')
recall = recall_score(Y_test, Y_pred, average='weighted')
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("LSTM Accuracy:", accuracy)
print("LSTM Precision:", precision)
print("LSTM Recall:", recall)
print("LSTM F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.8431 - loss: 0.3687 - val_accuracy: 0.9186 - val_loss: 0.2185
Epoch 2/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.8998 - loss: 0.2647 - val_accuracy: 0.9274 - val_loss: 0.1991
Epoch 3/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9053 - loss: 0.2506 - val_accuracy: 0.9248 - val_loss: 0.2024
Epoch 4/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9123 - loss: 0.2373 - val_accuracy: 0.9214 - val_loss: 0.2155
Epoch 5/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9132 - loss: 0.2352 - val_accuracy: 0.9308 - val_loss: 0.1970
Epoch 6/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9172 - loss: 0.2260 - val_accuracy: 0.9303 - val_loss: 0.1933
Epoch 7/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 16s 6ms/step - accuracy: 0.9183 - loss: 0.2232 - val_accuracy: 0.9326 - val_loss: 0.1886
Epoch 8/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 17s 6ms/step - accuracy: 0.9177 - loss: 0

In [48]:
import pandas as pd
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))
sample = X_test[random_index].reshape(1, 1, -1)  # Reshape to (1, timesteps=1, features)
true_label = np.array(Y_test)[random_index]

predicted_prob = model.predict(sample)[0, 0]
predicted_label = int(predicted_prob > 0.5)

print("\nRandomly selected sample:")
print(f"True label: {true_label}")
print(f"LSTM Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step

Randomly selected sample:
True label: 0
LSTM Predicted label (class): 0

Prediction time: 0.113623 seconds


In [49]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

for i in range(total_samples_processed):
    start_time = time.time()

    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, 1, -1)  # Reshape for LSTM (batch_size, timesteps, features)
    true_label = np.array(Y_test)[random_index]

    predicted_prob = model.predict(sample)[0, 0]
    predicted_label = int(predicted_prob > 0.5)

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("\nLSTM Model")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 93ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━

In [18]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class']).astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

lr = LogisticRegression(random_state=42)

lr.fit(X_train, Y_train)

Y_pred = lr.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("Logistic Regression Accuracy:", accuracy)
print("Logistic Regression Precision:", precision)
print("Logistic Regression Recall:", recall)
print("Logistic Regression F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Logistic Regression Accuracy: 0.6836467151097461
Logistic Regression Precision: 0.6363151597317098
Logistic Regression Recall: 0.6836467151097461
Logistic Regression F1-score: 0.5776355520934906

Training time: 7.690166 seconds


In [29]:
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = lr.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Logistic Regression Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 16
Logistic Regression Predicted label (class): 0

Prediction time: 0.000000 seconds


In [31]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = lr.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Logistic Regression")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Logistic Regression
Classification Accuracy: 0.69
Classification Precision (avg): 0.17
Classification Recall (avg): 0.17
Classification F1-Score (avg): 0.16
Average Prediction Time: 0.000051 seconds
Predictions with time >= 0.1s: 0 out of 500


In [36]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import warnings
import time

start_time = time.time()
warnings.filterwarnings("ignore", category=FutureWarning)

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class']).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

n_neighbor_amount = 10

knn = KNeighborsClassifier(n_neighbors=n_neighbor_amount)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
accuracy = accuracy_score(Y_test, Y_pred)
print("KNN Accuracy when n_neighbors =", n_neighbor_amount, ":", accuracy)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("KNN Precision:", precision)
print("KNN Recall:", recall)
print("KNN F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

KNN Accuracy when n_neighbors = 10 : 0.8053412240617275
KNN Precision: 0.7774265359277698
KNN Recall: 0.8053412240617275
KNN F1-score: 0.782508857918119

Training time: 1.773099 seconds


In [37]:
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = knn.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"KNN Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
KNN Predicted label (class): 0

Prediction time: 0.004595 seconds


In [40]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = knn.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("KNN")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

KNN
Classification Accuracy: 0.80
Classification Precision (avg): 0.54
Classification Recall (avg): 0.49
Classification F1-Score (avg): 0.50
Average Prediction Time: 0.002383 seconds
Predictions with time >= 0.1s: 0 out of 500


In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class']).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=20, random_state=42)

rf.fit(X_train, Y_train)

Y_pred = rf.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("Random Forest Accuracy:", accuracy)
print("Random Forest Precision:", precision)
print("Random Forest Recall:", recall)
print("Random Forest F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Random Forest Accuracy: 0.9331785152445876
Random Forest Precision: 0.9301850289220972
Random Forest Recall: 0.9331785152445876
Random Forest F1-score: 0.9302370120725711

Training time: 13.914604 seconds


In [55]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = rf.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Random Forest Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
Random Forest Predicted label (class): 0

Prediction time: 0.005921 seconds


In [56]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = rf.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Random Forest")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Random Forest
Classification Accuracy: 0.93
Classification Precision (avg): 0.89
Classification Recall (avg): 0.82
Classification F1-Score (avg): 0.84
Average Prediction Time: 0.003496 seconds
Predictions with time >= 0.1s: 0 out of 500


In [58]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class']).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.1, random_state=42)

dt = DecisionTreeClassifier(random_state=42)

dt.fit(X_train, Y_train)

Y_pred = dt.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("Decision Tree Accuracy:", accuracy)
print("Decision Tree Precision:", precision)
print("Decision Tree Recall:", recall)
print("Decision Tree F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Decision Tree Accuracy: 0.8956476140534871
Decision Tree Precision: 0.8964726731201722
Decision Tree Recall: 0.8956476140534871
Decision Tree F1-score: 0.8957646023959516

Training time: 5.192582 seconds


In [59]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = dt.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Decision Tree Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
Decision Tree Predicted label (class): 0

Prediction time: 0.004998 seconds


In [60]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = dt.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Decision Tree")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Decision Tree
Classification Accuracy: 0.90
Classification Precision (avg): 0.79
Classification Recall (avg): 0.80
Classification F1-Score (avg): 0.79
Average Prediction Time: 0.001738 seconds
Predictions with time >= 0.1s: 0 out of 500


In [62]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class']).astype(int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.31, random_state=42)

nb = GaussianNB()

nb.fit(X_train, Y_train)

Y_pred = nb.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("Naive Bayes Accuracy:", accuracy)
print("Naive Bayes Precision:", precision)
print("Naive Bayes Recall:", recall)
print("Naive Bayes F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Naive Bayes Accuracy: 0.6777826107969649
Naive Bayes Precision: 0.5963229929861497
Naive Bayes Recall: 0.6777826107969649
Naive Bayes F1-score: 0.6203884424277142

Training time: 0.212537 seconds


In [65]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
true_label = Y_test.iloc[random_index]

predicted_label = nb.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"Naive Bayes Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 6
Naive Bayes Predicted label (class): 5

Prediction time: 0.003015 seconds


In [66]:
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = pd.DataFrame([X_test.iloc[random_index]], columns=X_test.columns)
    true_label = Y_test.iloc[random_index]

    predicted_label = nb.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("Naive Bayes")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

Naive Bayes
Classification Accuracy: 0.68
Classification Precision (avg): 0.25
Classification Recall (avg): 0.22
Classification F1-Score (avg): 0.21
Average Prediction Time: 0.002105 seconds
Predictions with time >= 0.1s: 0 out of 500


C:\Users\david\AppData\Local\Temp\ipykernel_19500\1141856774.py:43: RuntimeWarning: invalid value encountered in divide
  precision = np.nan_to_num(tp / (tp + fp))
C:\Users\david\AppData\Local\Temp\ipykernel_19500\1141856774.py:45: RuntimeWarning: invalid value encountered in divide
  f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))


In [69]:
from sklearn.metrics import precision_score, recall_score, f1_score
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
import time

start_time = time.time()
features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features]
Y = (final_df['class']).astype(int)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.2, random_state=42)

mlp = MLPClassifier(hidden_layer_sizes=(100, 50), activation='relu', solver='adam', random_state=42,
                    max_iter=100, tol=0.001)

mlp.fit(X_train, Y_train)

Y_pred = mlp.predict(X_test)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("MLP Accuracy:", accuracy)
print("MLP Precision:", precision)
print("MLP Recall:", recall)
print("MLP F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

MLP Accuracy: 0.8671810622518541
MLP Precision: 0.8563174161226812
MLP Recall: 0.8671810622518541
MLP F1-score: 0.8573650139590068

Training time: 57.471611 seconds


C:\Users\david\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:699: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


In [70]:
import numpy as np
import pandas as pd
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))

sample = X_test[random_index].reshape(1, -1)
true_label = Y_test.iloc[random_index]

predicted_label = mlp.predict(sample)[0]

print("Randomly selected sample:")
print(f"True label: {true_label}")
print(f"MLP Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time
print(f"\nPrediction time: {elapsed_time:.6f} seconds")

Randomly selected sample:
True label: 0
MLP Predicted label (class): 0

Prediction time: 0.008230 seconds


In [71]:
import numpy as np
import time
import pandas as pd
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

if isinstance(Y_test, np.ndarray) and Y_test.ndim > 1:
    Y_test = np.argmax(Y_test, axis=1)

for i in range(total_samples_processed):
    start_time = time.time()
    
    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1)
    true_label = Y_test.iloc[random_index] if isinstance(Y_test, pd.Series) else Y_test[random_index]

    predicted_label = mlp.predict(sample)[0]

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.nan_to_num(tp / (tp + fp))
    recall = np.nan_to_num(tp / (tp + fn))
    f1 = np.nan_to_num(2 * (precision * recall) / (precision + recall))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("MLP Classification Results")
print(f"Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Precision (avg): {avg_precision:.2f}")
print(f"Recall (avg): {avg_recall:.2f}")
print(f"F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

MLP Classification Results
Accuracy: 0.86
Precision (avg): 0.72
Recall (avg): 0.65
F1-Score (avg): 0.66
Average Prediction Time: 0.000159 seconds
Predictions with time >= 0.1s: 0 out of 500


In [77]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, BatchNormalization, Dropout

start_time = time.time()

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features].values
Y = final_df['class'].astype(int).values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=15, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)

loss, accuracy = model.evaluate(X_test, Y_test)

precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("Multi-Class ANN Accuracy:", accuracy)
print("Multi-Class ANN Precision:", precision)
print("Multi-Class ANN Recall:", recall)
print("Multi-Class ANN F1-score:", f1)
print(f"\nTraining time: {elapsed_time:.6f} seconds")

Epoch 1/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.6807 - loss: 1.2295 - val_accuracy: 0.7780 - val_loss: 0.8026
Epoch 2/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7551 - loss: 0.8718 - val_accuracy: 0.7978 - val_loss: 0.7129
Epoch 3/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7707 - loss: 0.8110 - val_accuracy: 0.8039 - val_loss: 0.6904
Epoch 4/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7818 - loss: 0.7775 - val_accuracy: 0.8100 - val_loss: 0.6565
Epoch 5/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7865 - loss: 0.7501 - val_accuracy: 0.8173 - val_loss: 0.6456
Epoch 6/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7946 - loss: 0.7202 - val_accuracy: 0.8228 - val_loss: 0.6228
Epoch 7/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7989 - loss: 0.7043 - val_accuracy: 0.8281 - val_loss: 0.6038
Epoch 8/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7996 - loss: 0.7033 -

In [81]:
import pandas as pd
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))
sample = X_test[random_index].reshape(1, -1)
true_label = np.array(Y_test)[random_index]

predicted_probs = model.predict(sample)
predicted_label = np.argmax(predicted_probs)

print("\nRandomly selected sample:")
print(f"True label: {true_label}")
print(f"ANN Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step

Randomly selected sample:
True label: 9
ANN Predicted label (class): 9

Prediction time: 0.096434 seconds


In [86]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

for i in range(total_samples_processed):
    start_time = time.time()

    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1)
    true_label = np.array(Y_test)[random_index]

    predicted_probs = model.predict(sample)
    predicted_label = np.argmax(predicted_probs)

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

accuracy = np.mean(np.array(all_true_labels) == np.array(all_predicted_labels))
precision = precision_score(all_true_labels, all_predicted_labels, average='weighted', zero_division=1)
recall = recall_score(all_true_labels, all_predicted_labels, average='weighted', zero_division=1)
f1 = f1_score(all_true_labels, all_predicted_labels, average='weighted')

average_prediction_time = np.mean(prediction_times)

print("\nANN Model")
print(f"Classification Accuracy: {accuracy:.2f}")
print(f"Classification Precision (avg): {precision:.2f}")
print(f"Classification Recall (avg): {recall:.2f}")
print(f"Classification F1-Score (avg): {f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

In [94]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Input

start_time = time.time()

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features].values
Y = final_df['class'].astype(int).values

scaler = StandardScaler()
X = scaler.fit_transform(X)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("\nDeep Neural Network (DNN) Results")
print(f"Classification Accuracy: {accuracy:.2f}")
print(f"Classification Precision (avg): {precision:.2f}")
print(f"Classification Recall (avg): {recall:.2f}")
print(f"Classification F1-Score (avg): {f1:.2f}")
print(f"Training time: {elapsed_time:.6f} seconds")

Epoch 1/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.6907 - loss: 1.1991 - val_accuracy: 0.7685 - val_loss: 0.8128
Epoch 2/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7540 - loss: 0.8816 - val_accuracy: 0.7945 - val_loss: 0.7380
Epoch 3/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7676 - loss: 0.8211 - val_accuracy: 0.8078 - val_loss: 0.6871
Epoch 4/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.7814 - loss: 0.7704 - val_accuracy: 0.8126 - val_loss: 0.6644
Epoch 5/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7893 - loss: 0.7449 - val_accuracy: 0.8174 - val_loss: 0.6407
Epoch 6/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.7944 - loss: 0.7251 - val_accuracy: 0.8250 - val_loss: 0.6168
Epoch 7/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.7985 - loss: 0.7039 - val_accuracy: 0.8299 - val_loss: 0.6027
Epoch 8/10
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8036 - loss: 0.6849 -

In [99]:
import pandas as pd
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))
sample = X_test[random_index].reshape(1, -1)
true_label = np.array(Y_test)[random_index]

predicted_prob = model.predict(sample)[0, 0]
predicted_label = int(predicted_prob > 0.5)

print("\nRandomly selected sample:")
print(f"True label: {true_label}")
print(f"DNN Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

ValueError: Exception encountered when calling Sequential.call().

[1mCannot take the length of shape with unknown rank.[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=<unknown>, dtype=float32)
  • training=False
  • mask=None

In [97]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

for i in range(total_samples_processed):
    start_time = time.time()

    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, -1)
    true_label = np.array(Y_test)[random_index]

    predicted_prob = model.predict(sample)[0, 0]
    predicted_label = int(predicted_prob > 0.5)

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("\nDNN Model")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━

In [106]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, BatchNormalization, Dropout, Input

start_time = time.time()

features = ['posx', 'posy', 'spdx', 'spdy', 'spdx_n', 'spdy_n', 'aclx', 'acly', 'hedx', 'hedy', 'hedx_n', 'hedy_n']
X = final_df[features].values
Y = final_df['class'].astype(int).values  # Ensure integer labels (0-19)

scaler = StandardScaler()
X = scaler.fit_transform(X)

X = X.reshape(X.shape[0], 1, X.shape[1])  # Reshape for LSTM (samples, timesteps=1, features)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

model = Sequential([
    Input(shape=(1, X_train.shape[2])),
    LSTM(128, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(64, return_sequences=True, dropout=0.2, recurrent_dropout=0.1),
    BatchNormalization(),
    LSTM(32, dropout=0.2, recurrent_dropout=0.1),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(20, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_split=0.2)

Y_pred_prob = model.predict(X_test)
Y_pred = np.argmax(Y_pred_prob, axis=1)  # Get class with highest probability

accuracy = accuracy_score(Y_test, Y_pred)
precision = precision_score(Y_test, Y_pred, average='weighted', zero_division=1)
recall = recall_score(Y_test, Y_pred, average='weighted', zero_division=1)
f1 = f1_score(Y_test, Y_pred, average='weighted')

end_time = time.time()
elapsed_time = end_time - start_time

print("\nLSTM Multi-Class Results")
print(f"Classification Accuracy: {accuracy:.2f}")
print(f"Classification Precision (avg): {precision:.2f}")
print(f"Classification Recall (avg): {recall:.2f}")
print(f"Classification F1-Score (avg): {f1:.2f}")
print(f"Training time: {elapsed_time:.6f} seconds")

Epoch 1/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 26s 7ms/step - accuracy: 0.6557 - loss: 1.3935 - val_accuracy: 0.7326 - val_loss: 0.9643
Epoch 2/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7129 - loss: 1.0466 - val_accuracy: 0.7450 - val_loss: 0.8949
Epoch 3/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7249 - loss: 0.9896 - val_accuracy: 0.7528 - val_loss: 0.8572
Epoch 4/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7353 - loss: 0.9540 - val_accuracy: 0.7665 - val_loss: 0.8176
Epoch 5/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7398 - loss: 0.9351 - val_accuracy: 0.7739 - val_loss: 0.7981
Epoch 6/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - accuracy: 0.7463 - loss: 0.9106 - val_accuracy: 0.7816 - val_loss: 0.7768
Epoch 7/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7500 - loss: 0.8920 - val_accuracy: 0.7831 - val_loss: 0.7624
Epoch 8/15
2670/2670 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - accuracy: 0.7527 - loss: 0

In [111]:
import pandas as pd
import numpy as np
import time

start_time = time.time()

random_index = np.random.randint(0, len(X_test))
sample = X_test[random_index].reshape(1, 1, -1)
true_label = np.array(Y_test)[random_index]

predicted_prob = model.predict(sample)[0, 0]
predicted_label = int(predicted_prob > 0.5)

print("\nRandomly selected sample:")
print(f"True label: {true_label}")
print(f"LSTM Predicted label (class): {predicted_label}")

end_time = time.time()
elapsed_time = end_time - start_time

print(f"\nPrediction time: {elapsed_time:.6f} seconds")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step

Randomly selected sample:
True label: 7
LSTM Predicted label (class): 0

Prediction time: 0.077525 seconds


In [102]:
import pandas as pd
import numpy as np
import time
from sklearn.metrics import confusion_matrix

TIME_THRESHOLD = 0.1
total_samples_processed = 500
predictions_above_threshold = 0

all_true_labels = []
all_predicted_labels = []
prediction_times = []

for i in range(total_samples_processed):
    start_time = time.time()

    random_index = np.random.randint(0, len(X_test))

    sample = X_test[random_index].reshape(1, 1, -1)
    true_label = np.array(Y_test)[random_index]

    predicted_prob = model.predict(sample)[0, 0]
    predicted_label = int(predicted_prob > 0.5)

    all_true_labels.append(true_label)
    all_predicted_labels.append(predicted_label)

    end_time = time.time()
    elapsed_time = end_time - start_time
    prediction_times.append(elapsed_time)

    if elapsed_time >= TIME_THRESHOLD:
        predictions_above_threshold += 1

conf_matrix = confusion_matrix(all_true_labels, all_predicted_labels)

def calculate_metrics(cm):
    tp = np.diag(cm)
    fp = np.sum(cm, axis=0) - tp
    fn = np.sum(cm, axis=1) - tp

    precision = np.divide(tp, tp + fp, where=(tp + fp) != 0, out=np.zeros_like(tp, dtype=float))
    recall = np.divide(tp, tp + fn, where=(tp + fn) != 0, out=np.zeros_like(tp, dtype=float))

    f1 = np.divide(2 * precision * recall, precision + recall, where=(precision + recall) != 0, out=np.zeros_like(tp, dtype=float))

    return precision, recall, f1

precision_per_class, recall_per_class, f1_per_class = calculate_metrics(conf_matrix)
avg_precision = np.mean(precision_per_class)
avg_recall = np.mean(recall_per_class)
avg_f1 = np.mean(f1_per_class)

average_prediction_time = np.mean(prediction_times)

print("\nLSTM Model")
print(f"Classification Accuracy: {np.mean(np.array(all_true_labels) == np.array(all_predicted_labels)):.2f}")
print(f"Classification Precision (avg): {avg_precision:.2f}")
print(f"Classification Recall (avg): {avg_recall:.2f}")
print(f"Classification F1-Score (avg): {avg_f1:.2f}")
print(f"Average Prediction Time: {average_prediction_time:.6f} seconds")
print(f"Predictions with time >= {TIME_THRESHOLD}s: {predictions_above_threshold} out of {total_samples_processed}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━